In [1]:
import numpy as np
import pandas as pd
from ipywidgets import interact_manual, interact
from ipywidgets import Button, HBox, IntText, HTML, VBox
from IPython.display import display

# Chargement des données 

On charge les propositions présidentielles.

In [22]:
from funcs import load_data

In [23]:
propositions, candidate_labels = load_data()

# Quizz candidats 

In [24]:
class GUI:
    def __init__(self, candidate_labels, propositions):
        # internal vars
        self.candidate_labels = candidate_labels
        self.propositions = propositions
        self.choice = None
        self.clicked = dict(zip(self.candidate_labels, [False] * len(self.candidate_labels)))

        # candidate buttons
        buttons = [Button(description=label) for label in self.candidate_labels]
        for b in buttons:
            b.on_click(self.on_button_clicked)
        hbox1 = HBox()
        hbox1.children = buttons
        self.buttons = buttons

        # scorebox and new_question button
        self.scorebox = IntText(description='score', value=0)
        new_question = Button(description='nouvelle question !')
        new_question.on_click(self.create_new_question)
        hbox2 = HBox()
        hbox2.children = [self.scorebox, new_question]

        # proposition box
        self.html = HTML()

        # general layout
        vbox = VBox()
        vbox.children = [hbox1, hbox2, self.html]
        self.box = vbox

        # generate first question
        self.create_new_question()
        
    def create_new_question(self, b=None):
        choice = np.random.randint(len(self.candidate_labels))
        self.choice = choice
        current_propositions = self.propositions[choice]
        n = current_propositions.size
        self.html.value = '<p style="font-size: 150%;font-weight: 300;line-height: 1.39;margin: 0 0 12.5px;">{}</p>'.format(current_propositions[np.random.randint(n)])
        for b in self.buttons:
            b.style.button_color = None
            self.clicked = dict(zip(self.candidate_labels, [False] * len(self.candidate_labels)))
        
    def on_button_clicked(self, b):
        if not self.clicked[b.description]:
            if b.description == self.candidate_labels[self.choice]:
                self.scorebox.value += 1
                b.style.button_color = 'lightgreen'
            else:
                self.scorebox.value -= 1
                b.style.button_color = 'red'

        self.clicked[b.description] = True

In [25]:
gui = GUI(candidate_labels, propositions)
display(gui.box)

In [ ]:
def make_df(candidate, candidate_label):
    "Returns a dataframe with candidate name."
    ind_s = pd.Series(index=candidate.index, data=[candidate_label], name='candidat')
    return pd.DataFrame(data=[ind_s, candidate]).transpose()

In [ ]:
all_propositions = pd.concat([make_df(c, label) for c, label in zip(candidates, candidate_labels)])

In [ ]:
ind = all_propositions['proposition'].str.lower().str.contains('vert')

In [ ]:
all_propositions[ind]

In [ ]:
mask = all_propositions[ind]

In [ ]:
mask.groupby(by='candidat').sum().iloc[0].values

In [ ]:
from ipywidgets import interact, IntSlider, SelectMultiple

In [ ]:
mask.size

In [ ]:
slider = IntSlider(min=0, max=mask.size - 1)
slider

In [ ]:
selector = SelectMultiple(
        options=mask['proposition'].values.tolist(),
        value=[],
        description='',
        disabled=False,
        )
selector.layout.width='500px'
selector.layout.height = "300px"

In [ ]:
@interact
def search_for_props(search='croissance'):
    ind = all_propositions['proposition'].str.lower().str.contains(search)
    mask = all_propositions[ind]
    selector.options = mask['proposition'].values.tolist()

display(selector)